In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!pip install pyspark

update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode
     |████████████████████████████████| 204.2MB 67kB/s 
     |████████████████████████████████| 204kB 42.2MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=1a02061013bca0e50af911c6bd1a0cce9af78eca295a6cb5a2b5c42daadfb8a0
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [2]:
from pyspark import SparkFiles
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.ml.feature import *
from pyspark.ml.classification import *
from pyspark.ml import Pipeline
from pyspark.sql.functions import *
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [3]:
spark =  SparkSession.builder \
    .master("local[*]") \
    .config("spark.driver.memory", "14g") \
    .appName("CloudETLProject") \
    .getOrCreate()  

In [5]:
from google.colab import files
uploaded = files.upload()

Saving testfinal.csv to testfinal.csv
Saving trainfinal.csv to trainfinal.csv


In [13]:
import pandas as pd
pd_test = pd.read_csv('testfinal.csv', sep=',')
pd_train = pd.read_csv('trainfinal.csv', sep=',')
# testing = spark.read.csv(("drugsComTest.csv"), sep=",", header=True, timestampFormat="dd/mm/yyyy")
# df_train = spark.read.csv(("drugsComTrain.csv"), sep=",", header=True, timestampFormat="dd/mm/yyyy")

In [14]:
mySchema = StructType([ StructField("uniqueID", StringType(), True)\
                       ,StructField("drugName", StringType(), True)\
                       ,StructField("condition", StringType(), True)\
                       ,StructField("review", StringType(), True)\
                       ,StructField("rating", IntegerType(), True)\
                       ,StructField("date", StringType(), True)\
                       ,StructField("usefulCount", StringType(), True)\
                     ])

In [15]:
from pyspark.sql import SQLContext
from pyspark import SparkConf, SparkContext
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

df_test = sqlContext.createDataFrame(pd_test, schema=mySchema)
df_train = sqlContext.createDataFrame(pd_train, schema=mySchema)

In [16]:
spark.sparkContext.addFile("testfinal.csv")
spark.sparkContext.addFile("trainfinal.csv")


In [17]:
df_train.show()
# 1-5 as 0
# 6-10 as 1
# Can divide usefulCount by number of years to get an average 

+--------+--------------------+--------------------+--------------------+------+---------+-----------+
|uniqueID|            drugName|           condition|              review|rating|     date|usefulCount|
+--------+--------------------+--------------------+--------------------+------+---------+-----------+
|  206461|           Valsartan|Left Ventricular ...|it has no side ef...|     1|20-May-12|         27|
|   95260|          Guanfacine|                ADHD|my son is halfway...|     1|27-Apr-10|        192|
|   92703|              Lybrel|       Birth Control|i used to take an...|     0|14-Dec-09|         17|
|  138000|          Ortho Evra|       Birth Control|this is my first ...|     1| 3-Nov-15|         10|
|   35696|Buprenorphine / n...|   Opiate Dependence|suboxone has comp...|     1|27-Nov-16|         37|
|  155963|              Cialis|Benign Prostatic ...|nd day on mg star...|     0|28-Nov-15|         43|
|  165907|      Levonorgestrel|Emergency Contrac...|he pulled out but...|

In [18]:
df_test.show()

+--------+--------+--------------------+--------------------+------+---------+-----------+
|uniqueID|drugName|           condition|              review|rating|     date|usefulCount|
+--------+--------+--------------------+--------------------+------+---------+-----------+
|  103785| ZzzQuil|            Insomnia|gave me rapid hea...|     0|26-May-17|          1|
|   51804|   Zyvox|          Bacteremia|    it cured my mrsa|     1| 8-Oct-08|         20|
|   51790|   Zyvox|Skin and Structur...|i have been on zy...|     1| 3-Jul-15|          6|
|   51784|   Zyvox| Bacterial Infection|it didnt work as ...|     1|17-Apr-16|          1|
|   51797|   Zyvox|Methicillin-Resis...|i have had  major...|     1| 7-Dec-09|         22|
|   51791|   Zyvox|Methicillin-Resis...|i had mrsa inf la...|     1|31-Oct-13|         17|
|   51796|   Zyvox|Methicillin-Resis...|i got a mrsa stap...|     1| 2-Aug-11|         29|
|  222186|  Zytiga|     Prostate Cancer|very satisfied wi...|     1|23-Dec-16|          7|

In [19]:
drop_df = df_test.drop('uniqueID','drugName','condition', 'date', 'usefulCount' ).collect()
drop_df_two = df_train.drop('uniqueID','drugName','condition', 'date', 'usefulCount' ).collect()
# review_data = Tokenizer(inputCol="review", outputCol="words")


In [20]:
test_df = spark.createDataFrame(drop_df)
test_df.show()

+--------------------+------+
|              review|rating|
+--------------------+------+
|gave me rapid hea...|     0|
|    it cured my mrsa|     1|
|i have been on zy...|     1|
|it didnt work as ...|     1|
|i have had  major...|     1|
|i had mrsa inf la...|     1|
|i got a mrsa stap...|     1|
|very satisfied wi...|     1|
|effectiveness las...|     0|
|my psa was going ...|     1|
|on zytiga for  mo...|     1|
|began zytiga with...|     1|
|had tried clariti...|     1|
|this medicine wor...|     1|
|i have had cholin...|     1|
|after travelling ...|     1|
|i suffered from m...|     1|
|i recently had te...|     1|
|it works great fo...|     1|
|had hives nearly ...|     1|
+--------------------+------+
only showing top 20 rows



In [21]:
train_df = spark.createDataFrame(drop_df_two)
train_df.show()

+--------------------+------+
|              review|rating|
+--------------------+------+
|it has no side ef...|     1|
|my son is halfway...|     1|
|i used to take an...|     0|
|this is my first ...|     1|
|suboxone has comp...|     1|
|nd day on mg star...|     0|
|he pulled out but...|     0|
|abilify changed m...|     1|
| i ve had  nothin...|     0|
|i had been on the...|     1|
|i have been on th...|     1|
|i have taken anti...|     1|
|i had crohns with...|     0|
|have a little bit...|     0|
|started nexplanon...|     0|
|i have been takin...|     1|
|this drug worked ...|     1|
|ive been taking a...|     1|
|ive been on every...|     1|
|i have been on ta...|     1|
+--------------------+------+
only showing top 20 rows



In [22]:
train_df.groupBy("rating").count().orderBy(desc("count")).show()

+------+------+
|rating| count|
+------+------+
|     1|113209|
|     0| 48088|
+------+------+



In [23]:
test_df.groupby("rating").count().orderBy(desc("count")).show()


+------+-----+
|rating|count|
+------+-----+
|     1|37559|
|     0|16207|
+------+-----+



In [27]:
# add stop words
# add a stemmer before vectorizing 
# add stop words 
# try naive bayes as another option to compare

def build_trigrams(inputCol=["review","rating"], n=3):
    tokenizer = [Tokenizer(inputCol="review", outputCol="words")]

# Get rid of stop words
    stopremove = [StopWordsRemover(inputCol='words',outputCol='stop_tokens')]

# Stem the words

# Creates a column for every word, two and three words. n=3
    ngrams = [
        NGram(n=i, inputCol="stop_tokens", outputCol="{0}_grams".format(i))
        for i in range(1, n + 1)
    ]
# Min term frequency = how many times does it occur in review
# df - times drug occurs in document 
    cv = [
        CountVectorizer(vocabSize=2**14,inputCol="{0}_grams".format(i),
            outputCol="{0}_tf".format(i))
        for i in range(1, n + 1)
    ]
    idf = [IDF(inputCol="{0}_tf".format(i), outputCol="{0}_tfidf".format(i), minDocFreq=5) for i in range(1, n + 1)]
# cv and idf act as a 
    assembler = [VectorAssembler(
        inputCols=["{0}_tfidf".format(i) for i in range(1, n + 1)],
        outputCol="features"
    )]
# stringindexer 
    label_stringIdx = [StringIndexer(inputCol = "rating", outputCol = "label")]
    # selector = [ChiSqSelector(numTopFeatures=2**14,featuresCol='rawFeatures', outputCol="features")]
    lr = [LogisticRegression(maxIter=100)]
    return Pipeline(stages=tokenizer + stopremove + ngrams + cv + idf + assembler + label_stringIdx + lr)

In [25]:
trigram_pipelineFit = build_trigrams().fit(train_df)
test_results = trigram_pipelineFit.transform(test_df)

In [33]:
test_results.printSchema()

root
 |-- review: string (nullable = true)
 |-- rating: long (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- stop_tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- 1_grams: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- 2_grams: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- 3_grams: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- 1_tf: vector (nullable = true)
 |-- 2_tf: vector (nullable = true)
 |-- 3_tf: vector (nullable = true)
 |-- 1_tfidf: vector (nullable = true)
 |-- 2_tfidf: vector (nullable = true)
 |-- 3_tfidf: vector (nullable = true)
 |-- features: vector (nullable = true)
 |-- label: double (nullable = false)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [34]:
predictions.select(col("prediction"), col('label')).show(5)

+----------+-----+
|prediction|label|
+----------+-----+
|       1.0|  1.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
+----------+-----+
only showing top 5 rows



In [45]:
# classification report - look for false positive, false negative. 
# use a different evaluator to try
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator()

accuracy = predictions.filter(predictions.label == predictions.prediction).count() / float(predictions.count())
roc_auc = evaluator.evaluate(predictions)
print("Accuracy Score: {0:.4f}".format(accuracy))
print("ROC-AUC: {0:.4f}".format(roc_auc))

Accuracy Score: 0.9097
ROC-AUC: 0.3005


In [46]:
testreview = spark.createDataFrame([
                                    ("I sent these to my sister who is in pain daily. She was happy to get them and reported that they helped her a great deal.", 10)
], ["review", "rating"])

In [47]:
testreview = trigram_pipelineFit.transform(testreview)

In [48]:
testreview.select(["rawPrediction","probability", "prediction"]).show(truncate=False)

+--------------------------------------+-----------------------------------------+----------+
|rawPrediction                         |probability                              |prediction|
+--------------------------------------+-----------------------------------------+----------+
|[16.80364843162202,-16.80364843162202]|[0.9999999496188371,5.038116298628847E-8]|0.0       |
+--------------------------------------+-----------------------------------------+----------+



In [49]:
train_df.groupBy("rating").count().orderBy(desc("count")).show()

+------+------+
|rating| count|
+------+------+
|     1|113209|
|     0| 48088|
+------+------+



In [51]:
trigram_pipelineFit.save("test_save")

In [ ]:
# create a clean function to be applied to app.py for the input later on. 